### vi. Attacker relationship (how do they claim to know the victim, 1) met online? 2) friend of a friend?, 3) met the victim in person before)

#### Example sentences:
- I **got your contact during** one of my recent official trips to S/korea March/April this year as one of the  Economic Community of West African States (ECOWAS) Delegates accredited by the Korea Trade Promotion Agency (KOTRA) office to attend your  Trade Missions-the Korean International Commercial and Special Vehicle show tagged "CSV SHOW 2006" held there between Wednesday the 29th of March to Saturday Ist April 2006.
- I **got your contact from** a close Associate of mine who works with the Nigerian Chamber of Commerce and Industry  who visited your country for an International Trade Fair upon my quest for a trusted and reliable foreign businessman or company. 
- I GUESS THIS LETTER MAY COME TO YOU AS SURPRISE SINCE I **HAD NO PREVIOUS CORRESPONDENCE WITH YOU**.
- I know this letter will come to you as surprise. **I managed to get your contact line from** the Cote Divoire Chamber of Commerce, hence I did not waste time to contact you on a business proposal. 


#### Boolean Question Answering (Train a model from google dataset and use it in the email)

Model training code is taken from here: [Deep Learning has (almost) all the answers: Yes/No Question Answering with Transformers](https://medium.com/illuin/deep-learning-has-almost-all-the-answers-yes-no-question-answering-with-transformers-223bebb70189)

Trained using: `booleanQATraining.py`

#### Download model from google drive:

In [5]:
import gdown
import os

if not os.path.exists('Models'):
    os.makedirs('Models')

url = 'https://drive.google.com/uc?id=1_hqDsN9YoMPw20GyyrMdMuhYsTbOd1YR'
output = 'Models/booleanQA_batchSize8.pth'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1_hqDsN9YoMPw20GyyrMdMuhYsTbOd1YR
To: /Users/anthony/Documents/GitHub/DSCI-550-Assignment-1/notebooks/Models/booleanQA_batchSize8.pth
499MB [00:10, 47.3MB/s] 


'Models/booleanQA_batchSize8.pth'

In [19]:
import torch
import torch.nn as nn

class TempModel(nn.Module):
    def __init__(self):
        self.conv1 = nn.Conv2d(3, 5, (3, 3))
    def forward(self, inp):
        return self.conv1(inp)

model = TempModel()
model.load_state_dict(torch.load(file_path))
model.eval()

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'

In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
# Use a GPU if you have one available (Runtime -> Change runtime type -> GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def predict(question, passage):
    sequence = tokenizer.encode_plus(question, passage, return_tensors="pt")['input_ids'].to(device)
    logits = model(sequence)[0]
    probabilities = torch.softmax(logits, dim=1).detach().cpu().tolist()[0]
    proba_yes = round(probabilities[1], 2)
    proba_no = round(probabilities[0], 2)

    print(f"Question: {question}, Yes: {proba_yes}, No: {proba_no}")

In [18]:
passage_superbowl = """Super Bowl 50 was an American football game to determine the champion of the National Football League
                    (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated
                    the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title.
                    The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara,
                    California. As this was the 50th Super Bowl, the league emphasized the 'golden anniversary' with various
                    gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game
                    with Roman numerals (under which the game would have been known as 'Super Bowl L'), so that the logo could
                    prominently feature the Arabic numerals 50."""
 
passage_illuin = """Illuin designs and builds solutions tailored to your strategic needs using Artificial Intelligence
                  and the new means of human interaction this technology enables."""

superbowl_questions = [
"Did the Denver Broncos win the Super Bowl 50?", 
"Did the Carolina Panthers win the Super Bowl 50?",
"Was the Super Bowl played at Levi's Stadium?", 
"Was the Super Bowl 50 played in Las Vegas?", 
"Was the Super Bowl 50 played in February?", 
"Was the Super Bowl 50 played in March?"
]

question_illuin = "Is Illuin the answer to your strategic needs?"

for s_question in superbowl_questions:
    predict(s_question, passage_superbowl)

predict(question_illuin, passage_illuin)

TypeError: 'collections.OrderedDict' object is not callable

In [8]:
import json
import os
from ftfy import fix_text
import re
from transformers import pipeline

nlp = pipeline("question-answering")


PATH = '../data/separated by email/'

for email in os.listdir(PATH):
    print(email)
    
    j = json.load(open(os.path.join(PATH, email)))
    if 'X-TIKA:content' not in j:
        print(email, "does not have 'X-TIKA:content' key")
        continue
    text = j['X-TIKA:content']
    text_cleaned = fix_text(text).strip()
    text_cleaned = re.sub(r'(\n\s*)+', ' ', text_cleaned) # replace multiple newlines to single newline
    
    context = text_cleaned
    
    questions = [
        "How did I get your contact?",
        "How did we met?"
    ]
    
    booleanQuestions = [
        "Did we meet online?",
        "Did we meet in person"
    ]
    
    for question in questions:
        print(question)
        print(nlp(question=question, context=context))
        
    print('\n-------\n')
        

3721.json
How did I get your contact?
{'score': 0.30042335391044617, 'start': 2344, 'end': 2357, 'answer': 'telephone/fax'}
How did we met?
{'score': 0.24091817438602448, 'start': 2344, 'end': 2357, 'answer': 'telephone/fax'}


ValueError: only one element tensors can be converted to Python scalars

In [34]:
T5Tokenizer.from_pretrained('t5-base')

### ix. Attacker estimated age. You will use the USC Data Science AgePredictor, here: https://github.com/USCDataScience/AgePredictor